## QTM 350: Data Science Computing

### Assignment 08: SQL Practice with Python

### Due 02 April 2025

### Instructions:
Use Python's `sqlite3` library to complete the following tasks. Connect to an in-memory SQLite database or create a file named `assignment08.db`.

Ensure all your Python code and the query results (where applicable) are included in this notebook. Submit this `.ipynb` file on Canvas.

Begin by running the setup code cell to create and populate the `drivers` table.

### Setup: Create and Populate Database

In [2]:
import sqlite3
import pandas as pd # Often useful for displaying results

# Connect to the database (or create it)
conn = sqlite3.connect('assignment08.db') 
# conn = sqlite3.connect(':memory:') # Option for in-memory database
cursor = conn.cursor()
print("Database connected.")

# --- Create drivers Table ---
cursor.execute("DROP TABLE IF EXISTS drivers;")
create_table_sql = """
CREATE TABLE drivers(
    driver_id INT PRIMARY KEY,
    driver_name VARCHAR(30),
    nationality VARCHAR(15),
    team VARCHAR(30),
    victories INT,
    podiums INT,
    poles INT,
    points FLOAT
);
"""
cursor.execute(create_table_sql)
print("Table 'drivers' created.")

# --- Insert Data ---
drivers_data = [
    (1, 'Lewis Hamilton', 'British', 'Mercedes', 103, 182, 103, 4160.5),
    (2, 'Michael Schumacher', 'German', 'Ferrari', 91, 155, 68, 1566.0),
    (3, 'Ayrton Senna', 'Brazilian', 'McLaren', 41, 80, 65, 614.0),
    (4, 'Sebastian Vettel', 'German', 'Red Bull', 53, 122, 57, 3097.0),
    (5, 'Niki Lauda', 'Austrian', 'Ferrari', 25, 54, 24, 420.5),
    (6, 'Fernando Alonso', 'Spanish', 'Renault', 32, 101, 22, 2061.0),
    (7, 'Alain Prost', 'French', 'McLaren', 51, 106, 33, 798.5),
    (8, 'Charles Leclerc', 'Monegasque', 'Ferrari', 5, 21, 18, 699.0),
    (9, 'Max Verstappen', 'Dutch', 'Red Bull', 52, 92, 28, 2411.0),
    (10, 'Jenson Button', 'British', 'Brawn GP', 15, 50, 8, 1235.0),
    (11, 'Kimi Raikkonen', 'Finnish', 'Ferrari', 21, 103, 18, 1873.0),
    (12, 'Daniel Ricciardo', 'Australian', 'Red Bull', 8, 32, 3, 1274.0),
    (13, 'Valtteri Bottas', 'Finnish', 'Mercedes', 10, 67, 20, 1731.0),
    (14, 'Rubens Barrichello', 'Brazilian', 'Ferrari', 11, 68, 13, 658.0)
]

insert_sql = "INSERT INTO drivers VALUES (?, ?, ?, ?, ?, ?, ?, ?);"
cursor.executemany(insert_sql, drivers_data)
conn.commit()
print(f"{cursor.rowcount} records inserted into 'drivers'.")

# --- Verify Data Insertion ---
print("\n--- Initial Drivers Data ---")
df_initial = pd.read_sql_query("SELECT * FROM drivers;", conn)
print(df_initial)

Database connected.
Table 'drivers' created.
14 records inserted into 'drivers'.

--- Initial Drivers Data ---
    driver_id         driver_name nationality      team  victories  podiums  \
0           1      Lewis Hamilton     British  Mercedes        103      182   
1           2  Michael Schumacher      German   Ferrari         91      155   
2           3        Ayrton Senna   Brazilian   McLaren         41       80   
3           4    Sebastian Vettel      German  Red Bull         53      122   
4           5          Niki Lauda    Austrian   Ferrari         25       54   
5           6     Fernando Alonso     Spanish   Renault         32      101   
6           7         Alain Prost      French   McLaren         51      106   
7           8     Charles Leclerc  Monegasque   Ferrari          5       21   
8           9      Max Verstappen       Dutch  Red Bull         52       92   
9          10       Jenson Button     British  Brawn GP         15       50   
10         11      K

GitHub CoPilot was used in the develop of the code

1) List drivers ordered by `team` alphabetically and then by `victories` in descending order.

In [21]:
first_table = pd.read_sql_query("SELECT driver_id, driver_name, team, victories FROM drivers ORDER BY team ASC, victories DESC", conn)
print(first_table)

    driver_id         driver_name      team  victories
0          10       Jenson Button  Brawn GP         15
1           2  Michael Schumacher   Ferrari         91
2           5          Niki Lauda   Ferrari         25
3          11      Kimi Raikkonen   Ferrari         21
4          14  Rubens Barrichello   Ferrari         11
5           8     Charles Leclerc   Ferrari          5
6           7         Alain Prost   McLaren         51
7           3        Ayrton Senna   McLaren         41
8           1      Lewis Hamilton  Mercedes        103
9          13     Valtteri Bottas  Mercedes         10
10          4    Sebastian Vettel  Red Bull         53
11          9      Max Verstappen  Red Bull         52
12         12    Daniel Ricciardo  Red Bull          8
13          6     Fernando Alonso   Renault         32


2) List drivers who have won more races than the average number of victories across all drivers.

In [22]:
second_table = pd.read_sql_query("SELECT driver_id, driver_name, team, victories FROM drivers WHERE victories > (SELECT AVG(victories) FROM drivers) ORDER BY victories DESC", conn)
print(second_table)

   driver_id         driver_name      team  victories
0          1      Lewis Hamilton  Mercedes        103
1          2  Michael Schumacher   Ferrari         91
2          4    Sebastian Vettel  Red Bull         53
3          9      Max Verstappen  Red Bull         52
4          7         Alain Prost   McLaren         51
5          3        Ayrton Senna   McLaren         41


3) Find all drivers with more than 50 podiums OR who drive for the 'Ferrari' team.

In [23]:
third_table = pd.read_sql_query("SELECT driver_id, driver_name, team, podiums FROM drivers WHERE podiums > 50 OR team = 'Ferrari' ORDER BY podiums DESC", conn)
print(third_table)

    driver_id         driver_name      team  podiums
0           1      Lewis Hamilton  Mercedes      182
1           2  Michael Schumacher   Ferrari      155
2           4    Sebastian Vettel  Red Bull      122
3           7         Alain Prost   McLaren      106
4          11      Kimi Raikkonen   Ferrari      103
5           6     Fernando Alonso   Renault      101
6           9      Max Verstappen  Red Bull       92
7           3        Ayrton Senna   McLaren       80
8          14  Rubens Barrichello   Ferrari       68
9          13     Valtteri Bottas  Mercedes       67
10          5          Niki Lauda   Ferrari       54
11          8     Charles Leclerc   Ferrari       21


4) Count the number of drivers for each `nationality`, but only include drivers who have more than 40 `podiums`. Order the result by count in descending order.

In [24]:
fourth_table = pd.read_sql_query("SELECT nationality, COUNT(*) AS driver_count FROM drivers WHERE podiums > 40 GROUP BY nationality ORDER BY driver_count DESC", conn)
print(fourth_table)

  nationality  driver_count
0      German             2
1     Finnish             2
2     British             2
3   Brazilian             2
4     Spanish             1
5      French             1
6       Dutch             1
7    Austrian             1


5) Calculate the average `points` for each `team`, rounded to two decimal places.

In [25]:
fifth_table = pd.read_sql_query("SELECT team, ROUND(AVG(points), 2) AS average_points FROM drivers GROUP BY team ORDER BY average_points DESC", conn)
print(fifth_table)

       team  average_points
0  Mercedes         2945.75
1  Red Bull         2260.67
2   Renault         2061.00
3  Brawn GP         1235.00
4   Ferrari         1043.30
5   McLaren          706.25


6) List drivers who are in the top 20% based on their `points`.

In [26]:
sixth_table = pd.read_sql_query("WITH ranked_drivers AS (SELECT driver_name, team, points, NTILE(5) OVER (ORDER BY points DESC) AS percentile_rank FROM drivers ) SELECT driver_name, team, points FROM ranked_drivers WHERE percentile_rank = 1 ORDER BY points DESC", conn)
print(sixth_table)
print(drivers_data)

        driver_name      team  points
0    Lewis Hamilton  Mercedes  4160.5
1  Sebastian Vettel  Red Bull  3097.0
2    Max Verstappen  Red Bull  2411.0
[(1, 'Lewis Hamilton', 'British', 'Mercedes', 103, 182, 103, 4160.5), (2, 'Michael Schumacher', 'German', 'Ferrari', 91, 155, 68, 1566.0), (3, 'Ayrton Senna', 'Brazilian', 'McLaren', 41, 80, 65, 614.0), (4, 'Sebastian Vettel', 'German', 'Red Bull', 53, 122, 57, 3097.0), (5, 'Niki Lauda', 'Austrian', 'Ferrari', 25, 54, 24, 420.5), (6, 'Fernando Alonso', 'Spanish', 'Renault', 32, 101, 22, 2061.0), (7, 'Alain Prost', 'French', 'McLaren', 51, 106, 33, 798.5), (8, 'Charles Leclerc', 'Monegasque', 'Ferrari', 5, 21, 18, 699.0), (9, 'Max Verstappen', 'Dutch', 'Red Bull', 52, 92, 28, 2411.0), (10, 'Jenson Button', 'British', 'Brawn GP', 15, 50, 8, 1235.0), (11, 'Kimi Raikkonen', 'Finnish', 'Ferrari', 21, 103, 18, 1873.0), (12, 'Daniel Ricciardo', 'Australian', 'Red Bull', 8, 32, 3, 1274.0), (13, 'Valtteri Bottas', 'Finnish', 'Mercedes', 10, 67, 

7) Calculate the total `victories`, total `podiums`, and average `points` for drivers with 'German' `nationality`.

In [27]:
seventh_table = pd.read_sql_query("SELECT SUM(victories) AS total_victories, SUM(podiums) AS total_podiums, ROUND(AVG(points), 2) AS average_points FROM drivers WHERE nationality = 'German'", conn)
print(seventh_table)

   total_victories  total_podiums  average_points
0              144            277          2331.5


8) List all drivers whose `driver_name` contains both the letter 'a' and the letter 'e' (case-insensitive).

In [28]:
eighth_table = pd.read_sql_query("SELECT driver_name FROM drivers WHERE LOWER(driver_name) LIKE '%a%'AND LOWER(driver_name) LIKE '%e%'", conn)
print(eighth_table)

           driver_name
0       Lewis Hamilton
1   Michael Schumacher
2         Ayrton Senna
3     Sebastian Vettel
4      Fernando Alonso
5      Charles Leclerc
6       Max Verstappen
7       Kimi Raikkonen
8     Daniel Ricciardo
9      Valtteri Bottas
10  Rubens Barrichello


9) Retrieve the driver with the most `victories` within each `team`.

In [29]:
ninth_table = pd.read_sql_query("SELECT team, driver_name, MAX(victories) AS max_victories FROM drivers GROUP BY team", conn)
print(ninth_table)

       team         driver_name  max_victories
0  Brawn GP       Jenson Button             15
1   Ferrari  Michael Schumacher             91
2   McLaren         Alain Prost             51
3  Mercedes      Lewis Hamilton            103
4  Red Bull    Sebastian Vettel             53
5   Renault     Fernando Alonso             32


10) List drivers whose `points` are below the average `points` for their respective `team`.

In [30]:
tenth_table = pd.read_sql_query("SELECT driver_name, team, points FROM drivers WHERE points < (SELECT AVG(points) FROM drivers AS sub WHERE sub.team = drivers.team)", conn)
print(tenth_table)

          driver_name      team  points
0        Ayrton Senna   McLaren   614.0
1          Niki Lauda   Ferrari   420.5
2     Charles Leclerc   Ferrari   699.0
3    Daniel Ricciardo  Red Bull  1274.0
4     Valtteri Bottas  Mercedes  1731.0
5  Rubens Barrichello   Ferrari   658.0


11) Cleanup: Close the database connection

In [ ]:
conn.close()